In [1]:
from floodligt_functions import clean_metadata
from floodligt_functions import read_position_data_jsonl

import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
#from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile
from typing import Union, Tuple, Dict
from pathlib import Path
from floodlight.core.pitch import Pitch
from floodlight.io.utils import get_and_convert
from floodlight.core.teamsheet import Teamsheet
from floodlight.core.xy import XY
from floodlight.core.code import Code
from floodlight.models.kinematics import AccelerationModel
from floodlight.models.kinematics import DistanceModel
from floodlight.models.kinematics import VelocityModel

In [2]:
filepath_position = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/tracking-produced.jsonl")

In [3]:
filepath_metadata = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/rsc-bru_metadata.json")

In [4]:
filepath_cleaned_metadata = Path("cleaned_metadata.json")

In [5]:
meta_cleaned_path = Path("cleaned_metadata.json")

In [6]:
clean_metadata(filepath_metadata)

In [7]:
position_data = read_position_data_jsonl(
    filepath_position,
    filepath_cleaned_metadata
)

In [8]:
position_data

({'HT1': {'Home': XY(xy=array([[-46.38,  -0.06,  -0.57, ...,    nan,    nan,    nan],
          [-46.36,  -0.06,  -0.56, ...,    nan,    nan,    nan],
          [-46.35,  -0.08,  -0.55, ...,    nan,    nan,    nan],
          ...,
          [   nan,    nan,    nan, ...,    nan,    nan,    nan],
          [   nan,    nan,    nan, ...,    nan,    nan,    nan],
          [   nan,    nan,    nan, ...,    nan,    nan,    nan]],
         shape=(3054601, 42)), framerate=25, direction='lr'),
   'Away': XY(xy=array([[42.22, -0.24,  8.87, ...,   nan,   nan,   nan],
          [42.21, -0.24,  8.88, ...,   nan,   nan,   nan],
          [42.21, -0.23,  8.88, ...,   nan,   nan,   nan],
          ...,
          [  nan,   nan,   nan, ...,   nan,   nan,   nan],
          [  nan,   nan,   nan, ...,   nan,   nan,   nan],
          [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
         shape=(3054601, 42)), framerate=25, direction='rl'),
   'Ball': XY(xy=array([[-0.21,  0.07],
          [ 0.5 , -0.08],

In [9]:
# get list of home players
# this will be used to match player with correct position data
home_players = [player for player in position_data[3]['Home']["player"]]

In [10]:
# get list of away players
away_players = [player for player in position_data[3]['Away']["player"]]

In [11]:
HT1_possession = position_data[1]['HT1'].code

# Convert np.nan to None so Polars can handle them as nulls
HT1_possession = [x if isinstance(x, str) else None for x in HT1_possession]

# Create DataFrame
HT1_possession_df = pl.DataFrame({
    "HT1_possession": HT1_possession
})

In [12]:
HT2_possession = position_data[1]['HT2'].code
# Convert np.nan to None so Polars can handle them as nulls
HT2_possession = [x if isinstance(x, str) else None for x in HT2_possession]

# Create DataFrame
HT2_possession_df = pl.DataFrame({
    "HT2_possession": HT2_possession
})

In [13]:
HT1_Ball_Status = position_data[2]['HT1'].code
# Convert np.nan to None so Polars can handle them as nulls
HT1_Ball_Status = [x if isinstance(x, str) else None for x in HT1_Ball_Status]

# Create DataFrame
HT1_Ball_Status_df = pl.DataFrame({
    "HT1_Ball_Status": HT1_Ball_Status
})

In [14]:
HT2_Ball_Status = position_data[2]['HT2'].code
# Convert np.nan to None so Polars can handle them as nulls
HT2_Ball_Status = [x if isinstance(x, str) else None for x in HT2_Ball_Status]

# Create DataFrame
HT2_Ball_Status_df = pl.DataFrame({
    "HT2_Ball_Status": HT2_Ball_Status
})

In [15]:
# concatenation attempt to see if dataframes are actujally concatenable
pl.concat([HT1_possession_df, HT1_Ball_Status_df], how="horizontal")

HT1_possession,HT1_Ball_Status
str,str
"""H""","""D"""
"""H""","""D"""
"""H""","""D"""
"""A""","""A"""
"""A""","""A"""
…,…
null,null
null,null
null,null


In [16]:
xy_objects = position_data[0]

In [17]:
xy_objects['HT1']['Home']#.xy to access the xy values

XY(xy=array([[-46.38,  -0.06,  -0.57, ...,    nan,    nan,    nan],
       [-46.36,  -0.06,  -0.56, ...,    nan,    nan,    nan],
       [-46.35,  -0.08,  -0.55, ...,    nan,    nan,    nan],
       ...,
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan]],
      shape=(3054601, 42)), framerate=25, direction='lr')

In [18]:
xy_objects['HT1']['Home'].xy

array([[-46.38,  -0.06,  -0.57, ...,    nan,    nan,    nan],
       [-46.36,  -0.06,  -0.56, ...,    nan,    nan,    nan],
       [-46.35,  -0.08,  -0.55, ...,    nan,    nan,    nan],
       ...,
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan]],
      shape=(3054601, 42))

In [19]:
position_data

({'HT1': {'Home': XY(xy=array([[-46.38,  -0.06,  -0.57, ...,    nan,    nan,    nan],
          [-46.36,  -0.06,  -0.56, ...,    nan,    nan,    nan],
          [-46.35,  -0.08,  -0.55, ...,    nan,    nan,    nan],
          ...,
          [   nan,    nan,    nan, ...,    nan,    nan,    nan],
          [   nan,    nan,    nan, ...,    nan,    nan,    nan],
          [   nan,    nan,    nan, ...,    nan,    nan,    nan]],
         shape=(3054601, 42)), framerate=25, direction='lr'),
   'Away': XY(xy=array([[42.22, -0.24,  8.87, ...,   nan,   nan,   nan],
          [42.21, -0.24,  8.88, ...,   nan,   nan,   nan],
          [42.21, -0.23,  8.88, ...,   nan,   nan,   nan],
          ...,
          [  nan,   nan,   nan, ...,   nan,   nan,   nan],
          [  nan,   nan,   nan, ...,   nan,   nan,   nan],
          [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
         shape=(3054601, 42)), framerate=25, direction='rl'),
   'Ball': XY(xy=array([[-0.21,  0.07],
          [ 0.5 , -0.08],

Kinematics Function --> acceleration 

In [20]:
# Get the HT1 Home accelleration dataframe
am1 = AccelerationModel()
am1.fit(xy_objects['HT1']['Home'])
acceleration = am1.acceleration()
np_array = acceleration.property
acceleration_HT1_Home_df = pl.DataFrame(np_array, schema=home_players) 

In [21]:
# Get the HT2 Home accelleration dataframe
am2 = AccelerationModel()
am2.fit(xy_objects['HT2']['Home'])
acceleration = am2.acceleration()
np_array = acceleration.property
acceleration_HT2_Home_df = pl.DataFrame(np_array, schema=home_players) 

In [1]:
# Get the HT1 Away accelleration dataframe
am3 = AccelerationModel()
am3.fit(xy_objects['HT1']['Away'])
acceleration = am3.acceleration()
np_array = acceleration.property
acceleration_HT1_away_df = pl.DataFrame(np_array, schema=away_players) 

NameError: name 'AccelerationModel' is not defined

In [ ]:
# Get the HT2 Away accelleration dataframe
am4 = AccelerationModel()
am4.fit(xy_objects['HT2']['Away'])
acceleration = am4.acceleration()
np_array = acceleration.property
acceleration_HT2_away_df = pl.DataFrame(np_array, schema=away_players) 

Kinematics Function --> distance model

In [ ]:
dm = DistanceModel()
dm.fit(xy_objects['HT1']['Home'])
distance_df = dm.distance_covered()
np_array = distance_df.property
distance_HT1_Home_df = pl.DataFrame(np_array, schema=home_players)

NameError: name 'DistanceModel' is not defined

In [ ]:
dm = DistanceModel()
dm.fit(xy_objects['HT1']['Home'])
cumulative_distance_array = dm.cumulative_distance_covered()
np_array = cumulative_distance_array.property
cdistance_HT1_Home_df = pl.DataFrame(np_array, schema=home_players)

Kinematics Function --> velocity model

In [ ]:
vm = VelocityModel()
vm.fit(xy_objects['HT1']['Home'])
velocity_array = vm.velocity()
np_array = velocity_array.property
df = pl.DataFrame(np_array, schema=home_players)